### Word Group In Youtube

In [73]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [74]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [75]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # 200  # native word end index
unique_word_ratio = 70
word_usage_min = 20
word_usage_max = 500

# sentence ratio and time shift
adjust_sent_word_ratio = 30
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select 
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
sentence_word_count_number = 5   # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [76]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/1-Word Group In Youtube Sentence/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [77]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [78]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [79]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [80]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [81]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [82]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987921,karneleme,5
987922,karnaya,5
987923,dörtlümüzün,5
987924,karnavalınız,5


In [83]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [84]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
995,hale,70625
996,kavga,70611
997,arabayı,70310
998,oldun,70121


In [85]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [86]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [87]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

1000

#### Youtube Sentence Analysis

In [88]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [89]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [90]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,3,10.267,11.394,mumu,fI1BI4d5KFU
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [91]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [92]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,28.7,"staff, betimlemesi, dizinin, fi, lm","bu, tarafından"
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU,0.1,"betimleme, sesli, yaptırılmıştır, derneğine",
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU,0.1,wwwsebederorg,
3,3,10.267,11.394,mumu,fI1BI4d5KFU,0.1,mumu,
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,43.0,"katsayım, safhada, sinir, hat","bak, şu, an"
...,...,...,...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.9,"videoyu, danimarka, bulunmadığı, arazilerin, y...","şunu, da, bir, söylemek, olduğu, ki, isterim"
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.1,"işine, bisikletle, yarısına, ulaşımda, yakını,...","bu, gidiyor, insanların"
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,33.4,"danimarkayla, alakalı, şimdilik, söyleyeceklerim","bu, kadar"
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0,11.2,"unutmayın, beğenmeyi, abone, kanalıma, gittiys...",ve


In [93]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,28.7,"staff, betimlemesi, dizinin, fi, lm","bu, tarafından"
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU,0.1,"betimleme, sesli, yaptırılmıştır, derneğine",NaN
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU,0.1,wwwsebederorg,NaN
3,3,10.267,11.394,mumu,fI1BI4d5KFU,0.1,mumu,NaN
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,43.0,"katsayım, safhada, sinir, hat","bak, şu, an"
...,...,...,...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.9,"videoyu, danimarka, bulunmadığı, arazilerin, y...","şunu, da, bir, söylemek, olduğu, ki, isterim"
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.1,"işine, bisikletle, yarısına, ulaşımda, yakını,...","bu, gidiyor, insanların"
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,33.4,"danimarkayla, alakalı, şimdilik, söyleyeceklerim","bu, kadar"
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0,11.2,"unutmayın, beğenmeyi, abone, kanalıma, gittiys...",ve


In [94]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,43.0,"katsayım, safhada, sinir, hat","bak, şu, an"
1,6,18.687,19.854,otursana bir tanem,fI1BI4d5KFU,33.4,"otursana, tanem",bir
2,7,19.967,22.805,hah oturtmaya çalıştığına göre demek doğru,fI1BI4d5KFU,50.1,"çalıştığına, hah, oturtmaya","demek, doğru, göre"
3,8,23.921,25.737,doğru mu bu konuşsana ya,fI1BI4d5KFU,80.1,konuşsana,"mu, bu, doğru, ya"
4,10,30.479,31.550,ne,fI1BI4d5KFU,100.1,NaN,ne
...,...,...,...,...,...,...,...,...
3084978,3934200,590.325,594.700,zira bizim insanlarımız işine yaramayan ikinci...,dRssR_apbR0,40.1,"işine, zira, insanlarımız, yaramayan, kıyafetl...","yerine, el, ikinci, bizim"
3084979,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,50.1,"sahiplerine, ediyorlar, örneğidir, vermeyi, ih...","güzel, tercih, bir, çok, bu, ve"
3084980,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,43.9,"videoyu, danimarka, bulunmadığı, arazilerin, y...","şunu, da, bir, söylemek, olduğu, ki, isterim"
3084981,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,30.1,"işine, bisikletle, yarısına, ulaşımda, yakını,...","bu, gidiyor, insanların"


In [95]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
17,25,81.538,84.046,demek o yüzden öyle garip davrandın,fI1BI4d5KFU,83.4,davrandın,"öyle, demek, o, yüzden, garip",demek o yüzden öyle garip
29,38,135.778,139.810,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU,70.1,rezil,"kadın, deli, herkesin, etti, içinde, o, beni",o deli kadın herkesin içinde beni
34,43,147.019,149.009,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU,71.5,karşımdan,"böyle, ama, dedim, git, yapma",ama yapma böyle git dedim git
56,79,302.034,303.874,bana o kadar iyi geldi ki,fI1BI4d5KFU,100.1,NaN,"bana, geldi, kadar, ki, o, iyi",bana o kadar iyi geldi ki
59,83,311.418,313.642,oh oh yok yok yok,fI1BI4d5KFU,40.1,NaN,"oh, yok",oh oh yok yok yok
...,...,...,...,...,...,...,...,...,...
3084934,3934138,187.205,192.549,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0,66.8,"sevinir, geliyorsunuz, köyleri, görünce","insan, kadar, ki, o, sakin, dışarıda, hatta, hale",o kadar sakin ki dışarıda insan
3084975,3934197,578.076,581.389,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0,66.8,"sektörü, kıyafet","el, orada, ikinci, ciddi, var, yani",yani orada ciddi ciddi ikinci el
3084977,3934199,583.933,590.183,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0,60.1,"ülkemizde, dükkanı, kıyafet, hatırlamıyorum, g...","bence, varsa, da, el, çok, ikinci, eğer, iş, f...",eğer varsa da bence çok iş
3084979,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,50.1,"sahiplerine, ediyorlar, örneğidir, vermeyi, ih...","güzel, tercih, bir, çok, bu, ve",ve bu çok güzel bir


In [96]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81.538000,83.401086,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135.778000,138.062800,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,147.019000,148.549769,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,302.034000,303.874000,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,oh oh yok yok yok,311.418000,313.642000,oh oh yok yok yok,fI1BI4d5KFU
...,...,...,...,...,...
275379,o kadar sakin ki dışarıda insan,188.084392,190.316696,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
275380,yani orada ciddi ciddi ikinci el,578.076000,580.178481,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
275381,eğer varsa da bence çok iş,587.861571,589.528238,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
275382,ve bu çok güzel bir,597.441716,598.724012,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [97]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81.238000,83.701086,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135.478000,138.362800,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,146.719000,148.849769,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,301.734000,304.174000,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,oh oh yok yok yok,311.118000,313.942000,oh oh yok yok yok,fI1BI4d5KFU
...,...,...,...,...,...
275379,o kadar sakin ki dışarıda insan,187.784392,190.616696,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
275380,yani orada ciddi ciddi ikinci el,577.776000,580.478481,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
275381,eğer varsa da bence çok iş,587.561571,589.828238,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
275382,ve bu çok güzel bir,597.141716,599.024012,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [98]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,demek o yüzden öyle garip,81,84,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
1,o deli kadın herkesin içinde beni,135,138,o deli kadın herkesin içinde beni rezil etti h...,fI1BI4d5KFU
2,ama yapma böyle git dedim git,147,149,ama yapma böyle git dedim git karşımdan,fI1BI4d5KFU
3,bana o kadar iyi geldi ki,302,304,bana o kadar iyi geldi ki,fI1BI4d5KFU
4,oh oh yok yok yok,311,314,oh oh yok yok yok,fI1BI4d5KFU
...,...,...,...,...,...
275379,o kadar sakin ki dışarıda insan,188,191,hatta köyleri o kadar sakin ki dışarıda insan ...,dRssR_apbR0
275380,yani orada ciddi ciddi ikinci el,578,580,yani orada ciddi ciddi ikinci el kıyafet sektö...,dRssR_apbR0
275381,eğer varsa da bence çok iş,588,590,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
275382,ve bu çok güzel bir,597,599,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0


In [99]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
...,...,...,...,...,...
275379,o o o şey o,1183,1186,o o o şey o,_FsOnRDIJQQ
275380,o ne o o i,3947,3949,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ
275381,o ne o o i,3947,3949,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ
275382,o o o o o,595,597,o o o o o zamanlı dom dom yine yemek yapıyor n...,1JKejQ9pQBc


In [100]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,93
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,77
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,100
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,89
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,94
...,...,...,...,...,...,...
275379,o o o şey o,1183,1186,o o o şey o,_FsOnRDIJQQ,40
275380,o ne o o i,3947,3949,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ,60
275381,o ne o o i,3947,3949,o ne o o i lyasın yanındaki ayşen mi ya,D6Va3TXwFEQ,60
275382,o o o o o,595,597,o o o o o zamanlı dom dom yine yemek yapıyor n...,1JKejQ9pQBc,20


In [101]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,93
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,77
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,100
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,89
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,94
...,...,...,...,...,...,...
259419,i ki ay mı ay,1656,1658,i ki ay mı ay dile kolay,W2tWECa8dlo,80
259420,bir de o mu i,4254,4255,nereye çıkıyor bir de o mu i şten,Yr2ZLfWT0i8,100
259421,ne o da şu an,273,275,formasyonu ne o da şu an aktif hale getirdim d...,HgaAaBdkWks,100
259422,s o s ne iki,3259,3262,s o s ne iki adın mı var,rH5PqTOIt6Q,80


In [ ]:
### ADD search_string sentence len ratio filter

In [102]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_in_sent_result_repeat_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,herkesin kendi işini kendi yaptığı,zzjsW3gqurc
1,diye söylüyorum boş evde bunları,zzXTIIo7EvM
2,sahip yine aynı şekilde burada hem,zzXTIIo7EvM
3,geldik sona kaldı diye sanki bize biraz,zzOL9mRRU0c
4,kimsenin başına geldim artık hadi,zyhG7Z4EAoA
...,...,...
15604,merak etme kısa zamanda gelecek,05PK5asBrP0
15605,hoş geldiniz hoş bulduk efendim,05PK5asBrP0
15607,beni takip etmeye devam edin,01qjQ_KPgpY
15608,hepimiz biliyoruz ki şu ana kadar dünya,01qjQ_KPgpY


In [103]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_in_sent_result_repeat_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.drop(["unique_word_ratio"], axis=1, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
...,...,...,...,...,...
15605,boş ver be baba i,7310,7312,boş ver be baba i şin kaynağı sende değil mi,EFEFEcYl9es
15606,el ile el ele göz,1467,1469,kızımız yedi kat el ile el ele göz göze gelmiş,0cCZkBCbZco
15607,beş bin su on bin,4632,4635,elektrik yirmi beş bin su on bin,UDlsIakb9Kg
15608,e sağ ol be baba,3280,3282,e sağ ol be baba,vMxEQFO8teQ


In [104]:
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw
...,...,...,...,...,...
15359,boş ver be baba i,7310,7312,boş ver be baba i şin kaynağı sende değil mi,EFEFEcYl9es
15360,el ile el ele göz,1467,1469,kızımız yedi kat el ile el ele göz göze gelmiş,0cCZkBCbZco
15361,beş bin su on bin,4632,4635,elektrik yirmi beş bin su on bin,UDlsIakb9Kg
15362,e sağ ol be baba,3280,3282,e sağ ol be baba,vMxEQFO8teQ


In [32]:
#df_youtube_word_usage_merge[df_youtube_word_usage_merge[["search_string","video_id"]].duplicated()]

In [33]:
#df_youtube_word_usage_merge[(df_youtube_word_usage_merge["search_string"] == "hayır ben seni daha çok")]

In [105]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,başka biriyle herhangi bir şey yaşıyor musun k...,319,325,başka biriyle herhangi bir şey yaşıyor musun k...,KsZ_8oIQsv4,https://www.youtube.com/watch?v=KsZ_8oIQsv4&t=...
1,gidelim bir bakalım hep birlikte bakalım bir o...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,https://www.youtube.com/watch?v=eqkDognLtTc&t=...
2,nasıl olduğunu gördün zaten artık değil mi tek...,2614,2620,nasıl olduğunu gördün zaten artık değil mi tek...,N4YDaU51lvI,https://www.youtube.com/watch?v=N4YDaU51lvI&t=...
3,bir şekilde söylüyorum ve o zaman eminim ki ço...,4774,4781,bir şekilde söylüyorum ve o zaman eminim ki ço...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
4,benim kızım tamam bir şey yapmış hata yapmış o...,3090,3097,benim kızım tamam bir şey yapmış hata yapmış o...,CIQ3Ave21Kw,https://www.youtube.com/watch?v=CIQ3Ave21Kw&t=...
...,...,...,...,...,...,...
15359,boş ver be baba i,7310,7312,boş ver be baba i şin kaynağı sende değil mi,EFEFEcYl9es,https://www.youtube.com/watch?v=EFEFEcYl9es&t=...
15360,el ile el ele göz,1467,1469,kızımız yedi kat el ile el ele göz göze gelmiş,0cCZkBCbZco,https://www.youtube.com/watch?v=0cCZkBCbZco&t=...
15361,beş bin su on bin,4632,4635,elektrik yirmi beş bin su on bin,UDlsIakb9Kg,https://www.youtube.com/watch?v=UDlsIakb9Kg&t=...
15362,e sağ ol be baba,3280,3282,e sağ ol be baba,vMxEQFO8teQ,https://www.youtube.com/watch?v=vMxEQFO8teQ&t=...


In [106]:
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Youtube Videoid Text Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=word_group_adjust_select, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
    df_adjust_text_ratio = df_adjust_text_ratio[~(df_adjust_text_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
    
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
#df_word_group_time_loc_in_text_result.to_csv(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Adjust Result Analysis Multiprocess

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = True  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to all word group length

# # shared word frequency
# shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list3 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list3.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list3), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list4 = manager.list()
index_list = manager.list()
all_index_list = manager.list()
var_index_list = manager.list()
text_all_list = manager.list()  
df = df_adjust_text_ratio
df.reset_index(drop=True, inplace=True)
index_num = list(range(len(df)))
sent_len = word_group_adjust_select
sent_len_num = sentence_word_count_number 

def df_exract_word_group(index_num):    
    source_text = df.loc[index_num,"sentence"]
    opt_var = df.loc[index_num,"video_id"]
    words = re.findall(r"\w+", source_text, re.UNICODE)
    
    index_list = []
    for j in range(len(words)):
        if words[j] in word_list:
            index_list.append(j)
        else:
            pass
    all_index_list = []
    var_index_list = []
    for k in range(len(index_list)):
        try:
            var1 = index_list[k] + 1  
            var2 = index_list[k+1]
        except:
            var1 = index_list[k] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[k]
            var_index_list.append(var3)
        else:
            var3 = index_list[k]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    text_all_list = []
    for m in all_index_list:
        text_list = [] 
        for n in m:
            word = words[n]
            text_list.append(word)
            if sent_len:
                if len(text_list) >= sent_len_num:
                    text = " ".join(text_list)
                    text_all_list.append(text)
                else:
                    pass
            else:
               text = " ".join(text_list)
               text_all_list.append(text) 
        
    for search_string in text_all_list:
        #df_var["index"] = i
        #df_var["search_string"] = search_string
        #df_var["sentence"] = source_text
        #df_var["video_id"] = opt_var
        result_list4.append([index_num,search_string,source_text,opt_var])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(df_exract_word_group, index_num)

In [ ]:
df_adjust_text_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
df_adjust_text_ratio.sort_values(by="index", ascending=True, inplace=True)
df_adjust_text_ratio.reset_index(drop=True, inplace=True)
df_adjust_text_ratio.drop(["index"], axis=1, inplace=True)
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_All_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_28 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv")
df_28

In [ ]:
df_200 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_200

In [ ]:
df_1000 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv")
df_1000